<a href="https://colab.research.google.com/github/kentofujii/green_2025/blob/main/01_%E3%83%A2%E3%83%86%E3%82%99%E3%83%AA%E3%83%B3%E3%82%AF%E3%82%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# モデリング.
- 不均衡対応 : 重みづけ
- パラメータチューニング : oputuna

In [ ]:
try:
  from sktime.utils.plotting import plot_series
except:
  !pip install sktime
  from sktime.utils.plotting import plot_series

In [ ]:
trial_no = "1".zfill(3)
input_date = "20250608"
memo = "ベンチマーク"
n_splits = 5
early_stopping_rounds = 30
# lightgbm
param = {"objective":"regression",
          "metric":"rmse",
         "num_iterations":500,
          'random_seed':0,
         "verbosity":-1,
         "boosting_type":"gbdt",#"dart"#"gbdt"
         }
drop_cols = []

In [ ]:
# マウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
my_folder = "/content/drive/MyDrive/ドキュメント/コンペ/SIGNATE/SMBC_GREEN_2025"
os.listdir(my_folder)

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/ドキュメント/コンペ/utils")
from MyFunc import opt_dtype_df,func_time,HistBox, YYPlot

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error
import json
import pickle

In [ ]:
try:
  import optuna
  import lightgbm
  import optuna.integration.lightgbm as lgb
except:
  !pip install optuna optuna-integration lightgbm dask
  import optuna
  import lightgbm
  import optuna.integration.lightgbm as lgb


In [ ]:
fp_train = os.path.join(my_folder, f"data/inter/inter_train_{input_date}.parq")
fp_test = os.path.join(my_folder, f"data/inter/inter_test_{input_date}.parq")
fp_submit = os.path.join(my_folder, f"data/input/sample_submit.csv")
os.makedirs(os.path.join(my_folder, f"data/output/{trial_no}"), exist_ok=True)
fp_out = os.path.join(my_folder, f"data/output/{trial_no}/{trial_no}.csv")
fp_model = os.path.join(my_folder, f"data/output/{trial_no}/model.pickle")
fp_cond = os.path.join(my_folder, f"data/output/{trial_no}/cond.json")
fp_settings = os.path.join(my_folder, f"data/output/{trial_no}/settings.json")
settings_dict = {
    "fp_train" : fp_train,
    "fp_test" : fp_test,
    "memo" : memo,
    "n_splits" : n_splits,
    "early_stopping_rounds" : early_stopping_rounds,
    "param" : param,
    "drop_cols" : drop_cols
    }
assert (os.path.exists(fp_train))
assert (os.path.exists(fp_test))
assert (os.path.exists(fp_submit))
assert not os.path.exists(fp_out)

# データ準備

In [ ]:
df_train = pd.read_parquet(fp_train)
df_test = pd.read_parquet(fp_test)
df_train.shape, df_test.shape

In [ ]:
df_train.set_index("time", inplace=True)
df_test.set_index("time", inplace=True)
df_train.index = pd.PeriodIndex(df_train.index, freq="h")
df_test.index = pd.PeriodIndex(df_test.index, freq="h")

In [ ]:
target_col = "price_actual"
# log_target_col = "log_" + target_col
# df_train[log_target_col] = np.log1p(df_train[target_col])
# display(df_train[log_target_col].describe())
target_col

In [ ]:
HistBox(df_train[target_col])
# q25, q75 = df_train[target_col].quantile([0.25, 0.75])
# iqr = q75 - q25
# idx = (df_train[target_col] >= q25 - 1.5 * iqr) & (df_train[target_col] <= q75 + 1.5 * iqr)
df_train_raw = df_train.copy()
# df_train = df_train[idx]
HistBox(df_train[target_col])

In [ ]:
usecols = [x for x in df_train.columns if x not in ["index", target_col] + drop_cols ]
assert len(set(usecols)-set(df_test.columns))==0

In [ ]:
len(usecols), len(df_test.columns)

In [ ]:
X = df_train[usecols]
y = df_train[target_col]
X_test = df_test[usecols]

In [ ]:
# 実験
ts = TimeSeriesSplit(n_splits=n_splits)
for train_index, test_index in ts.split(X):
  print("学習:", X.index[train_index].min(), "~", X.index[train_index].max(), "検証:", X.index[test_index].min(), "~", X.index[test_index].max())

# 関数定義

In [ ]:
def tuning(X_train : np.ndarray, y_train : np.ndarray):
  # kf=KFold(n_splits=n_splits, shuffle=False, random_state=0)
  ts = TimeSeriesSplit(n_splits=n_splits)
  trainval = lgb.Dataset(X_train, label=y_train)

  call_backs = []
  if param["boosting_type"]!="dart":
    call_backs += [lightgbm.early_stopping(stopping_rounds=early_stopping_rounds)]
  # クロスバリデーションによるハイパーパラメータの探索
  tuner = lgb.LightGBMTunerCV(param, trainval, folds=ts, callbacks=call_backs, show_progress_bar=True)

  # ハイパーパラメータ探索の実行
  tuner.run()

  # サーチしたパラメータの表示
  best_params = tuner.best_params
  print("  Params: ")
  for key, value in best_params.items():
      print("    {}: {}".format(key, value))
  return best_params, tuner

In [ ]:
# パラメータ最適化
best_params, tuner = tuning(X, y)

In [ ]:
best_params

In [ ]:
test_size = len(df_train.loc["2017-01-01":])
# 最適パラメータでの評価
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, shuffle=False)
model = lightgbm.LGBMRegressor(**best_params)
call_backs = [lightgbm.early_stopping(stopping_rounds=early_stopping_rounds)]
model.fit(X_train, y_train, eval_set=[(X_train, y_train),(X_val, y_val)], callbacks=call_backs)
# learning curve
lightgbm.plot_metric(model)
plt.show()


In [ ]:
HistBox(y_train)
HistBox(y_val)

In [ ]:
y_pred = pd.Series(model.predict(X_val), index=X_val.index)
val_score = root_mean_squared_error(y_val, y_pred)
print("validation: ", val_score)

In [ ]:

YYPlot(y_pred, y_val)
# YYPlot(np.expm1(y_pred), np.expm1(y_val))

In [ ]:
plot_series(y_val, y_pred)

# submitファイル作成

In [ ]:
df_sample_submit = pd.read_csv(fp_submit, header=None)
df_sample_submit.head()

In [ ]:
model = lightgbm.LGBMRegressor(**best_params)
model.fit(X, y)


In [ ]:
y_test_pred = model.predict(X_test)
df_y_test = pd.DataFrame(zip(df_test.index, y_test_pred), columns=["index", target_col])
display(df_y_test.head())
assert len(df_y_test) == len(df_sample_submit)
df_y_test = df_sample_submit
df_y_test.iloc[:, 1] = y_test_pred
df_y_test.to_csv(fp_out, index=False, header=None)

In [ ]:
fp_settings

In [ ]:
cond_dict={"trial_no":trial_no, "input_date":input_date, "n_splits":n_splits, "early_stopping_rounds":early_stopping_rounds, "best_params":best_params, "val_score":val_score}
with open(fp_cond, "w") as f:
  json.dump(cond_dict, f)
with open(fp_model, "wb") as f:
  pickle.dump(model, f)
with open(fp_settings, "w", encoding="cp932") as f:
  json.dump(settings_dict, f)

In [ ]:
# prompt: modelを読み込む
# モデルの読み込み
with open(fp_model, "rb") as f:
  loaded_model = pickle.load(f)

# 読み込んだモデルで予測
y_test_pred_loaded = loaded_model.predict(X_test)
# 予測結果の比較
np.allclose(y_test_pred, y_test_pred_loaded)  # Trueが返れば、予測結果が一致


In [ ]:
X.columns